In [1]:
# -*- coding: utf-8 -*-
"""
Based on Yelp Fusion API code sample.

"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import pandas as pd

# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode



In [18]:

API_KEY= "tce8h_FqJw30vLO5EKIcuJ_hvnoHQ9H8D6408qWkwav4QkNo0lRLHLUQbG4hZtVafy4RStnjDP2R4HGv11-8S16bKaNCXDJ_1rdCzJZorapYft93jGU2r26S8pQrZHYx"


# API constants.
API_HOST = 'https://api.yelp.com'
#SEARCH_PATH = '/v3/categories'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
#DEFAULT_TERM = 'dinner'
#DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 50
#OFFSET = 950


In [3]:

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()




In [7]:
def search(api_key, term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        #'offset':OFFSET,
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


In [15]:
restaurants = search(API_KEY,"mexican restaurants", "OR")

Querying https://api.yelp.com/v3/businesses/search ...


In [16]:
restaurants

{'businesses': [{'id': 'Vp1kiliceNp6_K8tkRCTCg',
   'alias': 'la-cocina-restaurant-portland',
   'name': 'La Cocina Restaurant',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/xK5ZFyzJSzmCvvPgsSZFYw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/la-cocina-restaurant-portland?adjust_creative=TfNX6YDGX5itOpvpbnXurw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=TfNX6YDGX5itOpvpbnXurw',
   'review_count': 494,
   'categories': [{'alias': 'mexican', 'title': 'Mexican'}],
   'rating': 4.0,
   'coordinates': {'latitude': 45.550892, 'longitude': -122.667025},
   'transactions': ['pickup', 'delivery'],
   'price': '$',
   'location': {'address1': '3901 N Williams Ave',
    'address2': None,
    'address3': '',
    'city': 'Portland',
    'zip_code': '97227',
    'country': 'US',
    'state': 'OR',
    'display_address': ['3901 N Williams Ave', 'Portland, OR 97227']},
   'phone': '+15032785414',
   'display_phone': '(503) 278-5414',
   'dista

In [17]:
x = len(restaurants['businesses'])
x

50

In [23]:
search_categories =["mexican restaurants", "italian restaurants", "thai restaurants"]
restaurant_data = []
for search_category in search_categories:
    restaurants = search(API_KEY,search_category, "Portland, OR")
    x = len(restaurants['businesses'])
    for i in range(x):
        try:
            # Parse out name, review_count, categories, rating, price, latitude, longitude, city, state, country 
            restaurant_id = restaurants['businesses'][i]['id']
            restaurant_name = restaurants['businesses'][i]['name']
            restaurant_review_count = restaurants['businesses'][i]['review_count']
            categories = restaurants['businesses'][i]['categories']
            alias = [d.get('alias', None) for d in categories]
            if "italian" in alias:
                restaurant_category = "Italian"
            elif "mexican" in alias:
                restaurant_category = "Mexican"
            elif "thai" in alias:
                restaurant_category = "Thai"
            else:
                restaurant_category = "NA"
            restaurant_rating = restaurants['businesses'][i]['rating']
            restaurant_price = restaurants['businesses'][i]['price']
            restaurant_city = restaurants['businesses'][i]['location']['city']
            restaurant_country = restaurants['businesses'][i]['location']['country']
            restaurant_state = restaurants['businesses'][i]['location']['state']
            restaurant_lat = restaurants['businesses'][i]['coordinates']['latitude']
            restaurant_lng = restaurants['businesses'][i]['coordinates']['longitude']

            # Append the restaurant information into restaurant_data list
            restaurant_data.append({"id": restaurant_id,
                                    "name": restaurant_name,
                                    "review_count": restaurant_review_count,
                                    "category":restaurant_category,
                                    "rating": restaurant_rating,
                                    "price": restaurant_price,
                                    "city":restaurant_city,
                                    "state":restaurant_state,
                                    "country":restaurant_country,
                                    "Lat": restaurant_lat, 
                                    "Lng": restaurant_lng})  
        except:
            print("Skipping...")
            pass

Querying https://api.yelp.com/v3/businesses/search ...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Querying https://api.yelp.com/v3/businesses/search ...
Skipping...
Skipping...
Skipping...
Skipping...
Querying https://api.yelp.com/v3/businesses/search ...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...


In [24]:
restaurant_data


[{'id': 'Vp1kiliceNp6_K8tkRCTCg',
  'name': 'La Cocina Restaurant',
  'review_count': 494,
  'category': 'Mexican',
  'rating': 4.0,
  'price': '$',
  'city': 'Portland',
  'state': 'OR',
  'country': 'US',
  'Lat': 45.550892,
  'Lng': -122.667025},
 {'id': 'UZUjgF7XW7WkvyUb8_qCiA',
  'name': 'Matador NW Portland',
  'review_count': 872,
  'category': 'Mexican',
  'rating': 3.5,
  'price': '$$',
  'city': 'Portland',
  'state': 'OR',
  'country': 'US',
  'Lat': 45.5332,
  'Lng': -122.69846},
 {'id': 'B0fBpezDvj2Bp-8KOlMMRw',
  'name': 'Nuestra Cocina',
  'review_count': 613,
  'category': 'Mexican',
  'rating': 4.5,
  'price': '$$',
  'city': 'Portland',
  'state': 'OR',
  'country': 'US',
  'Lat': 45.50503,
  'Lng': -122.64379},
 {'id': 'Ir1GUgWHVsXpJeRTfk0HTQ',
  'name': "Carlita's - Portland",
  'review_count': 222,
  'category': 'NA',
  'rating': 4.0,
  'price': '$$',
  'city': 'Portland',
  'state': 'OR',
  'country': 'US',
  'Lat': 45.53172,
  'Lng': -122.68295},
 {'id': '3L-ezs0

In [26]:
restaurant_data_json = json.dumps(restaurant_data, indent=2)
with open("restaurant_data.json", "w") as outfile:
    outfile.write(restaurant_data_json)

In [25]:
# Convert the restaurant_data into a Pandas DataFrame
restaurant_data_df = pd.DataFrame(restaurant_data)

# Show Record Count
restaurant_data_df

,id,name,review_count,category,rating,price,city,state,country,Lat,Lng
0,Vp1kiliceNp6_K8tkRCTCg,La Cocina Restaurant,494,Mexican,4.0,$,Portland,OR,US,45.550892,-122.667025
1,UZUjgF7XW7WkvyUb8_qCiA,Matador NW Portland,872,Mexican,3.5,$$,Portland,OR,US,45.533200,-122.698460
2,B0fBpezDvj2Bp-8KOlMMRw,Nuestra Cocina,613,Mexican,4.5,$$,Portland,OR,US,45.505030,-122.643790
3,Ir1GUgWHVsXpJeRTfk0HTQ,Carlita's - Portland,222,NA,4.0,$$,Portland,OR,US,45.531720,-122.682950
4,3L-ezs0VANOtmdJDlbha1Q,Güero,625,Mexican,4.5,$$,Portland,OR,US,45.524462,-122.637035
...,...,...,...,...,...,...,...,...,...,...,...
126,IzVwtJ_3G6er8roSNMgtcg,Srisiamchai Thai Restaurant,178,Thai,4.0,$$,Gresham,OR,US,45.499420,-122.431278
127,mHhD2ICjrZvJ3bab1PwEzw,Thai PK Restaurant,258,Thai,4.0,$$,Portland,OR,US,45.478836,-122.673238
128,CPlbpYBShdxbrfwAazYETA,Thai Bloom,1264,Thai,4.0,$$,Beaverton,OR,US,45.491881,-122.809445
129,-1mlAuW6bX5FYsYhrrBRLQ,Thai Village,292,Thai,4.0,$$,Wilsonville,OR,US,45.302270,-122.766100


3


37.79979


[{'id': 'msT3LrLB4fhN04HYHuFsew',
  'name': 'Bella Trattoria',
  'review_count': 1794,
  'category': 'Italian',
  'rating': 4.5,
  'price': '$$',
  'city': 'San Francisco',
  'state': 'CA',
  'country': 'US',
  'Lat': 37.78136,
  'Lng': -122.46092107729817},
 {'id': 'QueFVMcMlT-6aZFv2M47mg',
  'name': 'Bottega',
  'review_count': 660,
  'category': 'Italian',
  'rating': 4.5,
  'price': '$$',
  'city': 'San Francisco',
  'state': 'CA',
  'country': 'US',
  'Lat': 37.75472,
  'Lng': -122.4212},
 {'id': '8dUaybEPHsZMgr1iKgqgMQ',
  'name': 'Sotto Mare',
  'review_count': 4853,
  'category': 'Italian',
  'rating': 4.5,
  'price': '$$',
  'city': 'San Francisco',
  'state': 'CA',
  'country': 'US',
  'Lat': 37.79979,
  'Lng': -122.40834}]